In [106]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

In [107]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [108]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2, random_state = 1234)

In [109]:
model = models.Sequential()
model.add(layers.experimental.preprocessing.RandomCrop(28, 28, input_shape=(28, 28, 1)))
model.add(layers.experimental.preprocessing.RandomFlip("horizontal"))

model.add(layers.Conv2D(32, (3, 3),
#                         activation='relu',
                        input_shape=(28, 28, 1),
                        padding = "same")
         )
model.add(layers.BatchNormalization())
model.add(layers.Activation("relu"))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3),
#                         activation='relu',
                        padding = "same")
         )
model.add(layers.BatchNormalization())
model.add(layers.Activation("relu"))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())

model.add(layers.Dense(256, activation = "relu"))
model.add(layers.Dense(10, activation="softmax"))

In [110]:
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_crop_5 (RandomCrop)  (None, 28, 28, 1)         0         
                                                                 
 random_flip_5 (RandomFlip)  (None, 28, 28, 1)         0         
                                                                 
 conv2d_36 (Conv2D)          (None, 28, 28, 32)        320       
                                                                 
 batch_normalization_27 (Bat  (None, 28, 28, 32)       128       
 chNormalization)                                                
                                                                 
 activation_26 (Activation)  (None, 28, 28, 32)        0         
                                                                 
 max_pooling2d_36 (MaxPoolin  (None, 14, 14, 32)       0         
 g2D)                                                

In [111]:
model.compile(optimizer = "adam",
             loss = "categorical_crossentropy",
             metrics = ["accuracy"],
             )

early_stopping = EarlyStopping(monitor = "val_accuracy", patience = 10, restore_best_weights = True)

In [112]:
model.fit(x_train, y_train,
                  batch_size = 256,
                  epochs = 100,
                  validation_data=(x_val, y_val),
                  callbacks=[early_stopping],
         )

Epoch 1/100
188/188 [==============================] - 37s 190ms/step - loss: 0.6288 - accuracy: 0.8012 - val_loss: 2.1336 - val_accuracy: 0.3299
Epoch 2/100
188/188 [==============================] - 34s 179ms/step - loss: 0.3243 - accuracy: 0.8839 - val_loss: 1.0772 - val_accuracy: 0.5788
Epoch 3/100
188/188 [==============================] - 4894s 26s/step - loss: 0.2861 - accuracy: 0.8956 - val_loss: 0.3027 - val_accuracy: 0.8909
Epoch 4/100
188/188 [==============================] - 31s 163ms/step - loss: 0.2570 - accuracy: 0.9060 - val_loss: 0.2728 - val_accuracy: 0.9002
Epoch 5/100
188/188 [==============================] - 31s 167ms/step - loss: 0.2332 - accuracy: 0.9146 - val_loss: 0.2633 - val_accuracy: 0.9053
Epoch 6/100
188/188 [==============================] - 29s 156ms/step - loss: 0.2172 - accuracy: 0.9209 - val_loss: 0.3222 - val_accuracy: 0.8811
Epoch 7/100
188/188 [==============================] - 30s 160ms/step - loss: 0.2041 - accuracy: 0.9252 - val_loss: 0.3435 -

In [113]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'ACCURACY: {test_acc * 100:.2f}%')

313/313 [==============================] - 2s 7ms/step - loss: 0.2978 - accuracy: 0.9202
ACCURACY: 92.02%
